In [108]:
!pip install transformers
!pip install shap

#imports
import numpy as np
import pandas as pd
# import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split

#transformers import
import transformers as tsfmr
from transformers import DistilBertTokenizerFast, DistilBertModel
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

import shap

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [109]:
model_trained = TFDistilBertForSequenceClassification.from_pretrained("/home/jskye99/AI4GoodE1/trained_distilbert_model") # change to googledrive path when running in google colab
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
df = pd.read_csv('https://raw.githubusercontent.com/AI4GoodE1/AI4GoodE1/main/fraud_email_preprocessed.csv')
X = list(df['Text'])
y = list(df['Class'])

Some layers from the model checkpoint at /home/jskye99/AI4GoodE1/trained_distilbert_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /home/jskye99/AI4GoodE1/trained_distilbert_model and are newly initialized: ['dropout_119']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [110]:
X_tokened = tokenizer(X)
email_len_poll = []
for x in range(len(X_tokened['input_ids'])):
    if len(X_tokened['input_ids'][x]) > 512:
        email_len_poll.append(X[x])
print(f'number of emails over 512 tokens: {len(email_len_poll)}\npercentage of dataset over 512 tokens: {len(email_len_poll) / len(X)}')
#print(email_len_poll[1])

Token indices sequence length is longer than the specified maximum sequence length for this model (676 > 512). Running this sequence through the model will result in indexing errors


number of emails over 512 tokens: 2018
percentage of dataset over 512 tokens: 0.19689725826909943


In [111]:
X_Train, X_Test, y_Train, y_Test = train_test_split(X,y,test_size=0.3,random_state=0)

In [112]:
classifier = tsfmr.pipeline('text-classification', model=model_trained, tokenizer=tokenizer, framework='tf',
                            config='/home/jskye99/AI4GoodE1/trained_distilbert_model') # change to googledrive path when running in google colab
classifier(X_Test[:2])

# test_email = tokenizer(X_Test[0], truncation=True, padding=True, return_tensors="tf")
# output_test = model_trained(test_email)
# print(output_test.logits)

[{'label': 'LABEL_0', 'score': 0.9998916387557983},
 {'label': 'LABEL_0', 'score': 0.9999232292175293}]

In [113]:
# explainer = shap.Explainer(classifier)
# shap_values = explainer(X_Test[:1])
# print(shap_values)
# shap.plots.text(shap_values[:,:,"LABEL_1"])

In [114]:
explainer = shap.Explainer(classifier)
shap_values = explainer(X_Test[:3])
# print(shap_values)
print(shap_values.base_values)
shap.plots.text(shap_values[:,:,"LABEL_0"])

  0%|          | 0/240 [00:00<?, ?it/s]

Partition explainer: 100%|██████████| 3/3 [00:28<00:00,  2.09s/it]

  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 4it [02:39, 53.16s/it]                       

[[0.7167455  0.        ]
 [0.81440771 0.        ]
 [0.53683919 0.        ]]



Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [115]:
test_email1 = "Hi team, let’s use my webex for our meeting at 3:15 again. Here is the link: https://rbcteams.webex.com/meet/test.test Thanks"
#test_email1 = tokenizer(test_email1, truncation=True, padding=True, return_tensors="tf")
test_emails = []
test_emails.append(test_email1)
print(test_emails)
print(X_Test[:1])
# output = model_trained(test_email1)
# print(f"trained: {output.logits}")
shap_values = explainer(test_emails)
print(shap_values)
shap.plots.text(shap_values[:,:,"LABEL_0"])

['Hi team, let’s use my webex for our meeting at 3:15 again. Here is the link: https://rbcteams.webex.com/meet/test.test Thanks']
['consulted  sidwill do  to whom should i send addresses  lauren  huma  ']


  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 2it [00:39, 39.78s/it]               

.values =
array([[[ 6.48165246e-03,  2.29106657e-03],
        [ 6.48165246e-03,  2.29106657e-03],
        [ 6.48165246e-03, -5.38323708e-02],
        [ 3.44780087e-03, -7.99215585e-03],
        [ 2.23512658e-03,  2.09088102e-03],
        [ 2.23512658e-03,  2.09088102e-03],
        [ 7.38837653e-04,  3.41824526e-03],
        [ 7.38837653e-04,  3.41824526e-03],
        [-3.28065856e-02,  4.09842668e-02],
        [-1.08403805e-01,  1.15092346e-01],
        [-6.51568092e-03,  1.78492535e-02],
        [-1.03572412e-02,  1.17918123e-02],
        [-1.03572412e-02,  1.29585210e-02],
        [ 6.24958941e-01, -4.02758977e-01],
        [ 5.17644165e-02, -1.59499580e-02],
        [ 3.32272388e-02, -2.64621982e-02],
        [ 3.32272388e-02, -2.63410648e-02],
        [ 1.67444345e-02, -2.06738521e-02],
        [ 5.00980966e-02, -5.48008286e-02],
        [ 2.46957528e-02, -2.66611042e-02],
        [-7.36818209e-04, -5.38476639e-04],
        [-7.36818209e-04, -5.38476639e-04],
        [-7.36818209e-

In [116]:
print(shap_values.values[0])
print(shap_values.feature_names[0])
for x in range(len(shap_values)):
  for y in range(len(shap_values.feature_names[x])):
    print(f'Token: "{shap_values.feature_names[x][y]}"   Weights: {shap_values.values[x][y]}')

[[ 6.48165246e-03  2.29106657e-03]
 [ 6.48165246e-03  2.29106657e-03]
 [ 6.48165246e-03 -5.38323708e-02]
 [ 3.44780087e-03 -7.99215585e-03]
 [ 2.23512658e-03  2.09088102e-03]
 [ 2.23512658e-03  2.09088102e-03]
 [ 7.38837653e-04  3.41824526e-03]
 [ 7.38837653e-04  3.41824526e-03]
 [-3.28065856e-02  4.09842668e-02]
 [-1.08403805e-01  1.15092346e-01]
 [-6.51568092e-03  1.78492535e-02]
 [-1.03572412e-02  1.17918123e-02]
 [-1.03572412e-02  1.29585210e-02]
 [ 6.24958941e-01 -4.02758977e-01]
 [ 5.17644165e-02 -1.59499580e-02]
 [ 3.32272388e-02 -2.64621982e-02]
 [ 3.32272388e-02 -2.63410648e-02]
 [ 1.67444345e-02 -2.06738521e-02]
 [ 5.00980966e-02 -5.48008286e-02]
 [ 2.46957528e-02 -2.66611042e-02]
 [-7.36818209e-04 -5.38476639e-04]
 [-7.36818209e-04 -5.38476639e-04]
 [-7.36818209e-04 -5.38476639e-04]
 [-7.36818209e-04 -5.38476639e-04]
 [-7.36818209e-04 -5.38476639e-04]
 [-7.36818209e-04 -5.38476639e-04]
 [-7.36818209e-04 -5.38476639e-04]
 [-7.36818209e-04 -5.38476639e-04]
 [-7.36818209e-04 -5